In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' ,STATUS and SPECIAL_CONSIDERATIONS.
application_df=application_df.drop(columns=['EIN','SPECIAL_CONSIDERATIONS','STATUS'])
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
NAME,19568
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
INCOME_AMT,9
ASK_AMT,8747
IS_SUCCESSFUL,2


In [5]:
# Look at name value counts for binning
name_counts= application_df['NAME'].value_counts()
name_counts

,count
NAME,
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700
WASHINGTON STATE UNIVERSITY,492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,408
...,...
OUR PERFECT STORM,1
FOUNTAIN THEATER,1
COMBAT VETERANS MOTORCYCLE ASSOCIATION 33-7,1


In [9]:
# Choose a cutoff value and create a list of name types to be replaced
# cut off amount > 5

# use the variable name `application_types_to_replace`
name_types_to_replace= list(name_counts[name_counts <=5].index)

#print(list(application_types_to_replace))

# Replace in dataframe
for app in name_types_to_replace:
   application_df['NAME'] = application_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

,count
NAME,
Other,31574
PARENT BOOSTER USA INC,1260
TOPS CLUB INC,765
UNITED STATES BOWLING CONGRESS INC,700


In [11]:
# Look at APPLICATION_TYPE value counts for binning
application_counts= application_df['APPLICATION_TYPE'].value_counts()
application_counts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [12]:
# Choose a cutoff value and create a list of application types to be replaced
# cut off amount > 500

# use the variable name `application_types_to_replace`
application_types_to_replace= list(application_counts[application_counts<500].index)

#print(list(application_types_to_replace))

# Replace in dataframe
for app in application_types_to_replace:
   application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [13]:
# Look at CLASSIFICATION value counts for binning
classCounts=application_df['CLASSIFICATION'].value_counts()
classCounts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C1248,1
C6100,1
C1820,1


In [14]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classCounts[classCounts>1000]

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883


In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# cut off chosen > 1000
# use the variable name `classifications_to_replace`
classifications_to_replace= list(classCounts[classCounts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [35]:
#generate our categorical list
application_cat=application_df.dtypes[application_df.dtypes=='object'].index.tolist()
application_cat

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [36]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_df=pd.get_dummies(application_df)
app_dummies_df.head()

,ASK_AMT,IS_SUCCESSFUL,NAME_Other,NAME_PARENT BOOSTER USA INC,NAME_TOPS CLUB INC,NAME_UNITED STATES BOWLING CONGRESS INC,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,True,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,True,False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,True,False,False,False,False,False,False,True,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,True,False,False,False,False,False,False,True,...,True,False,False,False,True,False,False,False,False,False


In [37]:
# Split our preprocessed data into our features and target arrays
X=app_dummies_df.drop(['IS_SUCCESSFUL'],axis='columns').values

y=app_dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features = len(X_train[0])
layer1=100
layer2=30
layer3=10

nn = tf.keras.models.Sequential()

#First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1,input_dim=input_features, activation='relu')
    )

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2,activation='sigmoid'))

# third  hidden layer
nn.add(tf.keras.layers.Dense(units=layer3,activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 100)                 │           4,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 30)                  │           3,030 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 10)                  │             310 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,851 (30.67 KB)

 Trainable params: 7,851 (30.67 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
# Train the model
fitnn= nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6817 - loss: 0.6077
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7277 - loss: 0.5509
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7277 - loss: 0.5491
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7289 - loss: 0.5426
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7320 - loss: 0.5419
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7274 - loss: 0.5399
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7346 - loss: 0.5339
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7341 - loss: 0.5359
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7337 - loss: 0.5308
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7292 - loss: 0.5375
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7273 - loss: 0.5395
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7336 - loss: 0.5393
Loss: 0.5393144488334656, Accuracy: 0.7336443066596985


In [46]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_optimization1.h5')

Trying with Random Forest

In [47]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


In [48]:
# create random forest classifier
rf_model= RandomForestClassifier(n_estimators=128, random_state=78)

# fitting the model
rf_model=rf_model.fit(X_train_scaled,y_train)

#evaluate the model
y_pred=rf_model.predict(X_test_scaled)
print(f'Random Forest Model Accuracy: {accuracy_score(y_test,y_pred)*100:.3f}')

Random Forest Model Accuracy: 72.268
